In [ ]:
!pip install wandb

In [ ]:
import numpy as np
from keras.datasets import fashion_mnist
import wandb

In [ ]:
def train_val_split(X, y, val_percent, seed=42):
  np.random.seed(seed)
  total_samples = len(X)
  total_val = int((val_percent / 100) * total_samples)

  idx = np.random.permutation(total_samples)
  X = X[idx]
  y = y[idx]

  X_val = X[:total_val]
  y_val = y[:total_val]

  X_train = X[total_val:]
  y_train = y[total_val:]

  return X_train, y_train, X_val, y_val


(X_train_1, y_train_1), (X_test, y_test) = fashion_mnist.load_data()

X_train_1 = X_train_1.reshape(len(X_train_1), 28 * 28) / 255.0
X_test  = X_test.reshape(len(X_test), 28 * 28) / 255.0
val_percentage = 10
X_train, y_train, X_val, y_val = train_val_split(X_train_1, y_train_1, val_percentage)

In [ ]:
def one_hot(y, num_classes=10):
  Y = np.zeros((len(y), num_classes))
  Y[np.arange(len(y)), y] = 1
  return Y

Y_train = one_hot(y_train, 10)
Y_val   = one_hot(y_val, 10)
Y_test  = one_hot(y_test, 10)

In [ ]:
class NeuralNet:
  def __init__(self, sizes, activation, w_init, w_decay):
    self.sizes = sizes
    self.w_init = w_init
    self.activation = activation
    self.w_decay = w_decay

    self.weights = []
    self.biases = []

    for i in range(1, len(sizes)):
        if self.w_init == "random":
            W = np.random.randn(sizes[i-1], sizes[i]) * 0.1
        elif self.w_init == "xavier":
            std = np.sqrt(2 / (sizes[i-1] + sizes[i]))
            W = np.random.randn(sizes[i-1], sizes[i]) * std
        b = np.zeros((1, sizes[i]))
        self.weights.append(W)
        self.biases.append(b)

    self.m_W = []
    self.m_B = []
    self.v_W = []
    self.v_B = []
    self.beta = 0.9
    self.beta2 = 0.999

    for i in range(len(self.weights)):
        self.m_W.append(np.zeros_like(self.weights[i]))
        self.m_B.append(np.zeros_like(self.biases[i]))
        self.v_W.append(np.zeros_like(self.weights[i]))
        self.v_B.append(np.zeros_like(self.biases[i]))

    self.a_count = 0
    self.t = 0

  def softmax(self, inputs):
    exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
    return probabilities

  def relu(self, Z):
    return np.maximum(0, Z)

  def tanh(self, Z):
    return np.tanh(Z)

  def sigmoid(self, Z):
    return 1 / (1 + np.exp(-Z))

  def relu_derivative(self, Z):
    return (Z > 0).astype(float)

  def tanh_derivative(self, Z):
    return 1 - np.tanh(Z) ** 2

  def sigmoid_derivative(self, Z):
    s = 1 / (1 + np.exp(-Z))
    return s * (1 - s)

  def forward(self, X):
    self.A = [X]
    self.Z = []

    A = X
    L = len(self.weights)

    for i in range(L):
      Z = np.dot(A, self.weights[i]) + self.biases[i]
      self.Z.append(Z)

      if i == L - 1:
        A = self.softmax(Z)
      else:
        if self.activation == "relu":
            A = self.relu(Z)
        elif self.activation == "tanh":
            A = self.tanh(Z)
        elif self.activation == "sigmoid":
            A = self.sigmoid(Z)

      self.A.append(A)

    return A

  def loss(self, Y_true, Y_pred):
    Y_pred = np.clip(Y_pred, 1e-7, 1 - 1e-7)
    return -np.sum(Y_true * np.log(Y_pred), axis=1)

  def loss_batches(self, X, Y, batch_size):
    total_samples = X.shape[0]
    total_loss = 0.0

    for i in range(0, total_samples, batch_size):
      Xb = X[i:i + batch_size]
      Yb = Y[i:i + batch_size]
      probs = self.forward(Xb)
      total_loss += np.sum(self.loss(Yb, probs))

    return total_loss / total_samples

  def accuracy(self, X, Y, batch_size):
    total_samples = X.shape[0]
    correct = 0

    for i in range(0, total_samples, batch_size):
      X_batch = X[i:i + batch_size]
      Y_batch = Y[i:i + batch_size]
      probs = self.forward(X_batch)
      y_true = np.argmax(Y_batch, axis=1)
      y_pred = np.argmax(probs, axis=1)
      correct += np.sum(y_true == y_pred)

    return correct / total_samples

  def backward(self, Y_true):
    total_samples = Y_true.shape[0]
    L = len(self.weights)

    self.dW = [0] * L
    self.db = [0] * L

    dZ = self.A[L] - Y_true

    for i in range(L - 1, -1, -1):
      A_prev = self.A[i]
      self.dW[i] = np.dot(A_prev.T, dZ) / total_samples
      self.dW[i] += self.w_decay * self.weights[i]
      self.db[i] = np.sum(dZ, axis=0, keepdims=True) / total_samples

      if i > 0:
        dA_prev = np.dot(dZ, self.weights[i].T)

        if self.activation == "relu":
          grad = self.relu_derivative(self.Z[i - 1])
        elif self.activation == "tanh":
          grad = self.tanh_derivative(self.Z[i - 1])
        elif self.activation == "sigmoid":
          grad = self.sigmoid_derivative(self.Z[i - 1])

        dZ = dA_prev * grad

  def sgd_step(self, lr):
    for i in range(len(self.weights)):
      self.weights[i] -= lr * self.dW[i]
      self.biases[i]  -= lr * self.db[i]

  def momentum(self, lr):
    for i in range(len(self.weights)):
      self.m_W[i] = self.beta * self.m_W[i] + (1 - self.beta) * self.dW[i]
      self.m_B[i] = self.beta * self.m_B[i] + (1 - self.beta) * self.db[i]
      self.weights[i] -= lr * self.m_W[i]
      self.biases[i]  -= lr * self.m_B[i]

  def nesterov(self, lr):
    for i in range(len(self.weights)):
      self.m_W[i] = self.beta * self.m_W[i] + (1 - self.beta) * self.dW[i]
      self.m_B[i] = self.beta * self.m_B[i] + (1 - self.beta) * self.db[i]
      self.weights[i] -= lr * (self.beta * self.m_W[i] + (1 - self.beta) * self.dW[i])
      self.biases[i]  -= lr * (self.beta * self.m_B[i] + (1 - self.beta) * self.db[i])

  def rmsprop(self, lr):
    for i in range(len(self.weights)):
      self.v_W[i] = self.beta * self.v_W[i] + (1 - self.beta) * (self.dW[i] ** 2)
      self.v_B[i] = self.beta * self.v_B[i] + (1 - self.beta) * (self.db[i] ** 2)
      self.weights[i] -= lr * self.dW[i] / (np.sqrt(self.v_W[i]) + 1e-8)
      self.biases[i]  -= lr * self.db[i] / (np.sqrt(self.v_B[i]) + 1e-8)

  def adam(self, lr, eps=1e-8):
    self.a_count += 1
    t = self.a_count

    for i in range(len(self.weights)):
      self.m_W[i] = self.beta * self.m_W[i] + (1 - self.beta) * self.dW[i]
      self.m_B[i] = self.beta * self.m_B[i] + (1 - self.beta) * self.db[i]

      self.v_W[i] = self.beta2 * self.v_W[i] + (1 - self.beta2) * (self.dW[i] ** 2)
      self.v_B[i] = self.beta2 * self.v_B[i] + (1 - self.beta2) * (self.db[i] ** 2)

      mW = self.m_W[i] / (1 - self.beta ** t)
      mB = self.m_B[i] / (1 - self.beta ** t)
      vW = self.v_W[i] / (1 - self.beta2 ** t)
      vB = self.v_B[i] / (1 - self.beta2 ** t)

      self.weights[i] -= lr * mW / (np.sqrt(vW) + eps)
      self.biases[i]  -= lr * mB / (np.sqrt(vB) + eps)

  def nadam(self, lr, eps=1e-8):
    self.t += 1
    t = self.t

    for i in range(len(self.weights)):
      self.m_W[i] = self.beta * self.m_W[i] + (1 - self.beta) * self.dW[i]
      self.m_B[i] = self.beta * self.m_B[i] + (1 - self.beta) * self.db[i]

      self.v_W[i] = self.beta2 * self.v_W[i] + (1 - self.beta2) * (self.dW[i] ** 2)
      self.v_B[i] = self.beta2 * self.v_B[i] + (1 - self.beta2) * (self.db[i] ** 2)

      mW = self.m_W[i] / (1 - self.beta ** t)
      mB = self.m_B[i] / (1 - self.beta ** t)
      vW = self.v_W[i] / (1 - self.beta2 ** t)
      vB = self.v_B[i] / (1 - self.beta2 ** t)

      mW2 = self.beta * mW + (1 - self.beta) * self.dW[i] / (1 - self.beta ** t)
      mB2 = self.beta * mB + (1 - self.beta) * self.db[i] / (1 - self.beta ** t)

      self.weights[i] -= lr * mW2 / (np.sqrt(vW) + eps)
      self.biases[i]  -= lr * mB2 / (np.sqrt(vB) + eps)

In [ ]:
def train():
  wandb.init()
  config = wandb.config
  hidden_layers = config.hidden_layers
  hidden_size = config.hidden_size

  sizes = [784]
  for _ in range(hidden_layers):
    sizes.append(hidden_size)
  sizes.append(10)

  epochs = config.epochs
  batch_size = config.batch_size
  lr = config.lr
  activation = config.activation
  weight_init = config.weight_init
  opt = config.optimizer
  weight_decay = config.weight_decay
  model = NeuralNet(sizes, activation, weight_init, weight_decay)
  total_samples = X_train.shape[0]

  for epoch in range(epochs):
    idx = np.random.permutation(total_samples)
    X_t = X_train[idx]
    Y_t = Y_train[idx]

    for i in range(0, total_samples, batch_size):
      Xb = X_t[i:i + batch_size]
      Yb = Y_t[i:i + batch_size]

      model.forward(Xb)
      model.backward(Yb)

      if opt == "sgd":
        model.sgd_step(lr)
      elif opt == "momentum":
        model.momentum(lr)
      elif opt == "nesterov":
        model.nesterov(lr)
      elif opt == "rmsprop":
        model.rmsprop(lr)
      elif opt == "adam":
        model.adam(lr)
      elif opt == "nadam":
        model.nadam(lr)

    train_loss = model.loss_batches(X_train, Y_train, batch_size)
    train_acc  = model.accuracy(X_train, Y_train, batch_size)
    val_loss  = model.loss_batches(X_val, Y_val, batch_size)
    val_acc   = model.accuracy(X_val, Y_val, batch_size)

    wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_acc",
        "goal": "maximize"
        },
    "parameters": {
        "epochs": {
            "values": [5, 10]
            },
        "hidden_layers": {
            "values": [3, 4, 5]
            },
        "hidden_size": {
            "values": [32, 64, 128]
            },
        "lr": {
            "values": [1e-3, 1e-4]
            },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
            },
        "batch_size": {
            "values": [16, 32, 64]
            },
        "weight_init": {
            "values": ["random", "xavier"]
            },
        "activation": {
            "values": ["sigmoid", "tanh", "relu"]
            },
        "weight_decay": {
            "values": [0.0, 0.0005, 0.5]
        }
    }
}
wandb.login()
sweep_id = wandb.sweep(sweep_config, project="Fashion MNIST Hyperparameter Tuning with wandb")
wandb.agent(sweep_id, train, count = 8)

Create sweep with ID: ig26q0pw
Sweep URL: https://wandb.ai/cl-chellappan-atri-ai/Fashion%20MNIST%20Hyperparameter%20Tuning%20with%20wandb/sweeps/ig26q0pw
